In [ ]:
pip install deepchem

In [ ]:
!pip install transformers datasets rdkit pandas scikit-learn torch

In [ ]:
import deepchem as dc
import pandas as pd

tasks, datasets, transformers = dc.molnet.load_delaney(featurizer='ECFP', split='random')
train_dataset, test_dataset, valid_dataset = datasets
train_dataset

<DiskDataset X.shape: (np.int64(902), np.int64(1024)), y.shape: (np.int64(902), np.int64(1)), w.shape: (np.int64(902), np.int64(1)), ids: ['O=C1N(COC(=O)C)C(=O)C(N1)(c2ccccc2)c3ccccc3' 'Cc1cncc(C)c1'
 'COc1ccc(C=O)cc1' ... 'OC1C=CC2C1C3(Cl)C(=C(Cl)C2(Cl)C3(Cl)Cl)Cl '
 'O=C1N(COC(=O)CCC)C(=O)C(N1)(c2ccccc2)c3ccccc3'
 'CSc1nc(nc(n1)N(C)C)N(C)C'], task_names: ['measured log solubility in mols per litre']>

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from deepchem.models import SklearnModel

rf_model = SklearnModel(model=RandomForestRegressor(n_estimators=500, random_state=0))
rf_model.fit(train_dataset)

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

y_true = test_dataset.y
y_pred = rf_model.predict(test_dataset)

rmse = np.sqrt(mean_squared_error(y_true, y_pred))
r2_score=r2_score(y_true, y_pred)

results = []
results.append({
    'Модель': 'RandomForest',
    'RMSE': rmse,
    'R2': r2_score
})

print(f'RMSE: {rmse}')
print(f'R2: {r2_score}')

RMSE: 0.6363632444531097
R2: 0.5761827523590601


In [ ]:
import joblib

joblib.dump(rf_model, 'rf_model.pkt')

['rf_model.pkt']

In [ ]:
!pip install transformers

In [ ]:
# Загрузка датасета ESOL
tasks, datasets, transformers = dc.molnet.load_delaney(featurizer='Raw', split=None)
dataset = datasets[0]

In [ ]:
from transformers import RobertaTokenizer, RobertaForSequenceClassification
tokenizer = RobertaTokenizer.from_pretrained("seyonec/ChemBERTa-zinc-base-v1")
model = RobertaForSequenceClassification.from_pretrained("seyonec/ChemBERTa-zinc-base-v1", num_labels=1, problem_type='regression')

def tokenize(smiles_list):
  return tokenizer(smiles_list, padding=True, truncation=True, return_tensors='pt', max_length=128)

smiles = list(dataset.ids)
logS = dataset.y
tokenized = tokenize(smiles)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at seyonec/ChemBERTa-zinc-base-v1 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import torch

# Прогнозирование
with torch.no_grad():
    model.eval()
    inputs = tokenized
    outputs = model(**inputs)
    predictions = outputs.logits.squeeze().cpu().numpy()

In [ ]:
import sklearn.metrics
# Реальные значения (logS)
labels = logS.flatten()

# Вычисление RMSE
rmse = np.sqrt(mean_squared_error(labels, predictions))
r2_score = sklearn.metrics.r2_score(labels, predictions)
results.append({
    'Модель': 'ChemBERTa (without pre-training)',
    'RMSE': rmse,
    'R2': r2_score
})

print(f"Test RMSE: {rmse}")
print(f'Test R2:{r2_score}')

Test RMSE: 0.9751514392811743
Test R2:0.04907967046785422


In [ ]:
result_df = pd.DataFrame(results)

display(result_df)

,Модель,RMSE,R2
0,RandomForest,0.636363,0.576183
1,ChemBERTa (without pre-training),0.975151,0.049080
